In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
qa_df = pd.read_parquet("../korean_embedding/data/qa.parquet")
qa_df.head()

,retrieval_gt,qid,query,generation_gt
0,[[c68a1103-7cb5-4f86-a46a-5f75fb88cfc2]],4ce77353-5441-4d2d-94e4-e837fcb19e25,일본 정부가 AI 위험을 줄이기 위해 어떤 조치를 취했나요?,"[일본 정부는 AI 위험을 줄이기 위해 기술적 해결책을 중요시하며, AI 글로벌 파..."
1,[[d55e6e70-8741-4063-9c1d-cc7357b9c268]],4ec41559-67d1-45aa-858e-9747b6fb6bbd,마이크로소프트가 프랑스에 투자한 금액은 얼마인가요?,[마이크로소프트가 프랑스에 투자한 금액은 40억 유로입니다.]
2,[[61097ef9-739d-488a-9ce4-802433a349ba]],fc74a5be-098b-4563-9bbe-bd1fc23f45df,ICML 2025 행사에서 다루는 주요 영역은 무엇인가요?,"[ICML 2025 행사에서는 머신러닝, 딥러닝, 최적화, 신뢰할 수 있는 머신러닝..."
3,[[581b4311-2236-44dd-80b0-e02c6906984b]],14408056-937f-44a1-878e-e73c3b5b5f09,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 무엇인가요?,[구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 재향군인 ·군인가족협회와 굿...
4,[[db2b35e9-4f60-400b-bcf2-6443e978f621]],68713781-37e8-44ac-ab49-8e57b9402aec,GPT-4o는 어떤 성능을 보여주었나요?,"[GPT-4o는 GPT-4 터보와 동일한 성능을 보여주었으며, 다국어, 오디오, 이..."


In [3]:
qa_df = qa_df[["query", "generation_gt"]]
qa_df.columns = ["question", "ground_truth"]
qa_df["ground_truth"] = qa_df["ground_truth"].apply(lambda x: x[0])
qa_df.head()

,question,ground_truth
0,일본 정부가 AI 위험을 줄이기 위해 어떤 조치를 취했나요?,"일본 정부는 AI 위험을 줄이기 위해 기술적 해결책을 중요시하며, AI 글로벌 파트..."
1,마이크로소프트가 프랑스에 투자한 금액은 얼마인가요?,마이크로소프트가 프랑스에 투자한 금액은 40억 유로입니다.
2,ICML 2025 행사에서 다루는 주요 영역은 무엇인가요?,"ICML 2025 행사에서는 머신러닝, 딥러닝, 최적화, 신뢰할 수 있는 머신러닝 ..."
3,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 무엇인가요?,구글이 AI 기회 기금의 1차 수혜자로 선정한 단체는 재향군인 ·군인가족협회와 굿윌...
4,GPT-4o는 어떤 성능을 보여주었나요?,"GPT-4o는 GPT-4 터보와 동일한 성능을 보여주었으며, 다국어, 오디오, 이미..."


In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

/Users/anpigon/Library/Caches/pypoetry/virtualenvs/autorag-tutorial-ko-GYvpjDIx-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

corpus_df = pd.read_parquet("../korean_embedding/data/corpus.parquet")
documents = [
    Document(page_content=row["contents"], metadata=row["metadata"], id=row["doc_id"])
    for index, row in corpus_df.iterrows()
]
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3

In [6]:
from langchain_chroma import Chroma

vectordb = Chroma(
    persist_directory="../korean_embedding/benchmark_ollama/resources/chroma",
    embedding_function=embeddings,
)
vector_retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [30]:
bm25_retriever.invoke(qa_df["question"][0])

[Document(metadata={'creation_date': '2024-07-10', 'file_name': 'SPRi AI Brief_6월호_산업동향 최종.pdf', 'file_path': '/Users/anpigon/Workspace-ai/AutoRAG-tutorial-ko/99-Projects/korean_embedding/raw_docs/SPRi AI Brief_6월호_산업동향 최종.pdf', 'file_size': 4347658, 'file_type': 'application/pdf', 'last_modified_date': '2024-07-10', 'last_modified_datetime': datetime.datetime(2024, 7, 10, 1, 31, 20, 96983), 'next_id': None, 'page_label': '8', 'prev_id': '6d904c23-8262-4824-95f0-61064b530cc1'}, page_content='공통의 목적을 위해 협력할 것”이라고 설명\n∙오픈AI의 샘 알트만 (Sam Altman) CEO는 온라인 행사에 참석해 글로벌 AI 거버넌스에서 히로시마 \nAI 프로세스 하의 국제 지침과 행동 규범의 중요성을 강조하고 남반구 국가가 대거 참여한 프렌즈 \n그룹의 역할에 기대감을 표시\nn일본 정부는 AI 위험을 줄이기 위한 기술적 해결책도 중요하다는 점에서 도쿄에 국제협의체 ‘AI \n글로벌 파트너십 (GPAI, Global Partnership on AI)’ 센터를 설립하고 기술 연구와 테스트를 \n지원할 예정 \n∙일본 정부는 생성 AI로 인한 허위 정보와 같은 위험을 해결하기 위해 콘텐츠 출처 검증 기술의 개발 \n노력도 지원할 계획\n☞ 출처:  Ministry of Foreign Affairs of Japan, Prime Minister Kishida’s attendance at the Side Event on Generative AI at \nthe OECD Minister

In [33]:
vector_retriever.invoke(qa_df["question"][0])

[]

In [7]:
from langchain.retrievers import EnsembleRetriever

retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3],
    search_type="mmr",
)

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """단락을 읽고 질문에 답하세요.

# 질문
{query} 

# 단락
{retrieved_contents} 

# 답변
"""
)


def get_rag_chain(model="gemma2"):
    return (
        {
            "retrieved_contents": retriever,
            "query": RunnablePassthrough(),
        }
        | prompt
        | ChatOllama(model=model, temperature=0.1)
        | StrOutputParser()
    )

In [9]:
from datasets import Dataset


def get_data_generator(qa_df: DataFrame, model="gemma2") -> Dataset:
    llm = get_rag_chain(model)
    qa_df["answer"] = qa_df["question"].apply(lambda x: llm.invoke(x))
    qa_df["contexts"] = qa_df["question"].apply(
        lambda x: [d.page_content for d in retriever.invoke(x)]
    )
    return qa_df

In [15]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [18]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from ragas import RunConfig, evaluate
from ragas.llms import LangchainLLMWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_upstage import UpstageEmbeddings


langchain_llm = LangchainLLMWrapper(
    # ChatGoogleGenerativeAI(
    #     model="gemini-1.5-pro",
    #     temperature=0,
    #     max_tokens=None,
    #     timeout=None,
    #     max_retries=50,
    # ),
    ChatOpenAI(
        model="gpt-4o",
        temperature=0.1,
        max_tokens=None,
        timeout=None,
        max_retries=50,
    ),
    RunConfig(
        timeout=None,
        max_retries=50,
    ),
)


# embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "mps"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)


def ragas_evaluate(dataset: Dataset):
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]

    for m in metrics:
        m.__setattr__("llm", langchain_llm)
        if hasattr(m, "embeddings"):
            m.__setattr__("embeddings", embeddings)

    return evaluate(
        dataset,
        metrics=metrics,
        raise_exceptions=False,
    )

In [13]:
qa_df = qa_df.sample(10)
qa_df

,question,ground_truth
9,오픈AI의 슈퍼 얼라인먼트 팀이 해체된 이유는 무엇인가요?,오픈AI의 슈퍼 얼라인먼트 팀이 해체된 이유는 얼라인먼트 팀이 안전 연구에 소홀해질...
27,모델 성능이 최대 수준에 도달하기 위해 어떤 조건이 충족되어야 하는가?,모델 성능이 최대 수준에 도달하기 위해서는 컨텍스트 창 내의 예제가 수십만 개 토큰...
15,AI안전보안이사회는 어떤 목적으로 설립되었나요?,AI안전보안이사회는 국토안보부가 관할하는 주요 기반시설에서 AI의 안전하고 책임있는...
19,AI 자율제조 테스트 베드를 구축하는 목적은 무엇인가요?,AI 자율제조 테스트 베드를 구축하는 목적은 기업들이 AI 자율제조 시스템 구축 과...
8,구글 딥마인드가 어떤 방법을 이용하여 LLM의 성능을 향상시키는 연구를 진행했나요?,구글 딥마인드는 '상황 내 학습(ICL)'을 이용해 미세조정 없이 LLM의 성능을 ...
7,네이버가 2024년에 어떤 신규 모델을 출시했나요?,네이버는 2024 년에 '대시(HCX-DASH)' 모델을 출시했습니다.
17,미국 기업의 AI 사용률은 어떻게 변화했나요?,미국 기업의 AI 사용률은 2023년 9월 3.7%에서 2024년 2월에 5.4%로...
4,GPT-4o는 어떤 성능을 보여주었나요?,"GPT-4o는 GPT-4 터보와 동일한 성능을 보여주었으며, 다국어, 오디오, 이미..."
21,GPT-4o의 응답시간은 어떻게 되나요?,"GPT-4o의 응답시간은 최소 0.23초, 평균 0.32초입니다."
24,'합성 콘텐츠로 인한 위험 감소(NIST AI 100-4)' 지침은 무엇을 안내하고...,'합성 콘텐츠로 인한 위험 감소(NIST AI 100-4)' 지침은 AI로 생성되거...


In [20]:
gemma2_df = ragas_evaluate(
    Dataset.from_pandas(get_data_generator(qa_df, model="gemma2"))
)
gemma2_df.to_pandas().to_parquet("gemma2.parquet")
gemma2_df

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'faithfulness': 0.8364, 'answer_relevancy': 0.2980, 'context_recall': 0.9000, 'context_precision': 0.8000}

In [21]:
qwen2_df = ragas_evaluate(Dataset.from_pandas(get_data_generator(qa_df, model="qwen2")))
qwen2_df.to_pandas().to_parquet("qwen2.parquet")

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [22]:
mistral_df = ragas_evaluate(
    Dataset.from_pandas(get_data_generator(qa_df, model="mistral"))
)
mistral_df.to_pandas().to_parquet("mistral.parquet")

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [23]:
eeve_df = ragas_evaluate(
    Dataset.from_pandas(get_data_generator(qa_df, model="EEVE-Korean-10.8B"))
)
eeve_df.to_pandas().to_parquet("eeve.parquet")

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [30]:
df_result = pd.merge(
    pd.merge(
        pd.DataFrame(list(gemma2_df.items()), columns=["Metric", "Gemma2"]),
        pd.DataFrame(list(qwen2_df.items()), columns=["Metric", "Qwen2"]),
        on="Metric",
    ),
    pd.merge(
        pd.DataFrame(list(mistral_df.items()), columns=["Metric", "Mistral"]),
        pd.DataFrame(list(eeve_df.items()), columns=["Metric", "EEVE"]),
        on="Metric",
    ),
    on="Metric",
)
df_result

,Metric,Gemma2,Qwen2,Mistral,EEVE
0,faithfulness,0.836372,0.757908,0.914544,0.916667
1,answer_relevancy,0.297962,0.313162,0.345320,0.500535
2,context_recall,0.900000,0.800000,0.800000,0.800000
3,context_precision,0.800000,0.800000,0.800000,0.800000
